In [1]:
from ru_test import dataset, labels
from ru_xlm_roberta_model import *

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [2]:
label_to_id = {label: label_id for label_id, label in enumerate(labels)}
id_to_label = {label_id: label for label_id, label in enumerate(labels)}

In [3]:
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.constraints.pre_transformation import (
    InputColumnModification,
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder

from textattack.attack_recipes import TextFoolerJin2019
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.transformations import WordSwapEmbedding
from textattack.shared.attack import Attack

from textattack.shared import WordEmbedding

In [4]:
import numpy as np
import pickle

embedding_matrix = np.load('ru_fasttext_30000/embeddings_matrix.npy')
nn_matrix = np.load('ru_fasttext_30000/nn_matrix.npy')
index2word = pickle.load(open('ru_fasttext_30000/index2word.pcl', 'rb'))
word2index = pickle.load(open('ru_fasttext_30000/word2index.pcl', 'rb'))

In [5]:
import collections

smart_word2index = collections.defaultdict(int, word2index)
smart_index2word = collections.defaultdict(str, index2word)

In [6]:
ru_fasttext_embedding = WordEmbedding(embedding_matrix, word2index, index2word, nn_matrix)

In [7]:
def build_textfooler(model):
    #
    # Swap words with their 50 closest embedding nearest-neighbors.
    # Embedding: Counter-fitted PARAGRAM-SL999 vectors.
    #
    transformation = WordSwapEmbedding(max_candidates=30, embedding=ru_fasttext_embedding)
    #
    # Don't modify the same word twice or the stopwords defined
    # in the TextFooler public implementation.
    #
    # fmt: off
    stopwords = set(
    #    ["a", "about", "above", "across", "after", "afterwards", "again", "against", "ain", "all", "almost", "alone", "along", "already", "also", "although", "am", "among", "amongst", "an", "and", "another", "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are", "aren", "aren't", "around", "as", "at", "back", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "both", "but", "by", "can", "cannot", "could", "couldn", "couldn't", "d", "didn", "didn't", "doesn", "doesn't", "don", "don't", "down", "due", "during", "either", "else", "elsewhere", "empty", "enough", "even", "ever", "everyone", "everything", "everywhere", "except", "first", "for", "former", "formerly", "from", "hadn", "hadn't", "hasn", "hasn't", "haven", "haven't", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "i", "if", "in", "indeed", "into", "is", "isn", "isn't", "it", "it's", "its", "itself", "just", "latter", "latterly", "least", "ll", "may", "me", "meanwhile", "mightn", "mightn't", "mine", "more", "moreover", "most", "mostly", "must", "mustn", "mustn't", "my", "myself", "namely", "needn", "needn't", "neither", "never", "nevertheless", "next", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "o", "of", "off", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "per", "please", "s", "same", "shan", "shan't", "she", "she's", "should've", "shouldn", "shouldn't", "somehow", "something", "sometime", "somewhere", "such", "t", "than", "that", "that'll", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "this", "those", "through", "throughout", "thru", "thus", "to", "too", "toward", "towards", "under", "unless", "until", "up", "upon", "used", "ve", "was", "wasn", "wasn't", "we", "were", "weren", "weren't", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "with", "within", "without", "won", "won't", "would", "wouldn", "wouldn't", "y", "yet", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves"]
    )
    # fmt: on
    constraints = [RepeatModification(), StopwordModification(stopwords=stopwords)]
    #
    # During entailment, we should only edit the hypothesis - keep the premise
    # the same.
    #
    input_column_modification = InputColumnModification(
        ["premise", "hypothesis"], {"premise"}
    )
    constraints.append(input_column_modification)
    # Minimum word embedding cosine similarity of 0.5.
    # (The paper claims 0.7, but analysis of the released code and some empirical
    # results show that it's 0.5.)
    #
    
    constraints.append(WordEmbeddingDistance(min_cos_sim=0.5))
    
    #
    # Only replace words with the same part of speech (or nouns with verbs)
    #
    
    #constraints.append(PartOfSpeech(allow_verb_noun_swap=True))
    
    #
    # Universal Sentence Encoder with a minimum angular similarity of ε = 0.5.
    #
    # In the TextFooler code, they forget to divide the angle between the two
    # embeddings by pi. So if the original threshold was that 1 - sim >= 0.5, the
    # new threshold is 1 - (0.5) / pi = 0.840845057
    #
    use_constraint = UniversalSentenceEncoder(
        threshold=0.84,
        metric="angular",
        compare_against_original=False,
        window_size=15,
        skip_text_shorter_than_window=True,
    )
    constraints.append(use_constraint)
    #
    # Goal is untargeted classification
    #
    goal_function = UntargetedClassification(model)
    #
    # Greedily swap words with "Word Importance Ranking".
    #
    search_method = GreedyWordSwapWIR(wir_method="delete")

    return Attack(goal_function, constraints, transformation, search_method)

In [8]:
#recipe = TextFoolerJin2019(
#    goal_function=UntargetedClassification(model), 
#    search_method=GreedyWordSwapWIR(wir_method="delete"),
#    transformation=WordSwapEmbedding(max_candidates=50),
#)
#recipe.constraints = constraints

In [9]:
import pandas as pd

result_df = pd.DataFrame({'attack_result': [], 'cross_val_batch': []})

In [10]:
dataset_names = [f'ru_train_crossval_{fold_id}' for fold_id in range(5)]
model_names = [f'xlm_roberta_model_ru_crossval_{fold_id}' for fold_id in range(5)]

In [ ]:
from tqdm import tqdm

for batch_id, (dataset_name, model_name) in enumerate(zip(dataset_names, model_names)):
    cur_dataset = __import__(dataset_name).dataset
    cur_model = __import__(model_name).model
    attacker = build_textfooler(cur_model)
    for idx, result in tqdm(enumerate(attacker.attack_dataset(cur_dataset))):
        result_df = result_df.append({
            'attack_result': result.__str__(color_method='html'),
            'cross_val_batch': batch_id
        }, ignore_index=True)
        #print(('x' * 20), f'Result {idx+1}', ('x' * 20))
        #print(result.__str__(color_method='html'))
        #print(len(str(result).split('\n\n')))
    result_df.to_csv('new_ru_xlm_roberta_cross_val_result_30.csv')
    del cur_model


10 LABELS


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [ ]:
print(len(result_df))

In [ ]:
def remove_color_tags(text):
    text = re.sub(r'<font color = [a-zA-Z]+>', ' ', text)
    text = re.sub(r'</font>', ' ', text)
    text = text.replace('  ', ' ')
    return text

In [ ]:
all_targets = []

for dataset_name in dataset_names:
    dataset_labels = [id_to_label[pair[1]] for pair in __import__(dataset_name).dataset]
    all_targets = all_targets + dataset_labels
print(all_targets)

In [ ]:
import re

ru_attacked_df = pd.DataFrame()
attack_count = collections.defaultdict(int)
word_count = collections.defaultdict(list)
good_count = 0

for raw_text, real_target in zip(result_df.attack_result, all_targets):
    if len(raw_text.split('\n\n')) == 3:
        header, old, new = raw_text.split('\n\n')
        _, label_id1, label_id2 = re.split(r'<font color = [a-zA-Z]+>', header)
        label1 = id_to_label[int(label_id1.split()[0])]
        label2 = id_to_label[int(label_id2.split()[0])]
        changed_words = len(tuple(re.finditer(r'<font color = [a-zA-Z]+>', old)))
        word_count[(label1, label2)].append(changed_words)
        attack_count[(label1, label2)] += 1
        
        
        ru_attacked_df = ru_attacked_df.append(
            {'old_text': remove_color_tags(old), 
             'text': remove_color_tags(new),
             'changed_words_num': changed_words,
             'old_model_target': label1,
             'new_model_target': label2,
             'target': real_target
            },
            ignore_index=True
        )
        good_count += 1
print(good_count)

clean_stat_df = pd.DataFrame()
for (label1, label2) in attack_count.keys():
    cur_attack_count = attack_count[(label1, label2)]
    cur_word_count = word_count[(label1, label2)]
    clean_stat_df = clean_stat_df.append(
        {'model_label_old': label1, 
         'model_label_new': label2, 
         'mean_words': sum(cur_word_count) / cur_attack_count, 
         'median': np.median(cur_word_count)
        }, 
        ignore_index=True
    )
clean_stat_df.head()

In [20]:
print(good_count)

718


In [ ]:
clean_stat_df.to_csv('/home/mlepekhin/data/new_ru_textfooler_stat_30.csv')

In [ ]:
ru_attacked_df.head()

In [ ]:
ru_attacked_df.to_csv('/home/mlepekhin/data/new_ru_attacked_30.csv')

In [ ]:
remove_color_tags('<font color = gray>Нижнекамскнефтехим')